# Scraping Kompas.com

Pada python notebook ini akan dijelaskan mengenai scraping artikel dalam kompas.com

Scraping dimulai dari halaman awal, lalu akan menelusuri setiap link, dan akan mengambil artikel beserta properties lainnya.


### Inisialisasi Url Lib

urllib merupakan modul python untuk mengakses halaman web. Digunakan fungsi urlopen dan read untuk membaca kode HTML. Dapat dilihat r merupakan string HTML.

In [ ]:
import urllib

r = urllib.urlopen('http://tekno.kompas.com/').read()
print(type(r))
print(r[1:100])


### Inisialisasi BeautifulSoup

Untuk membantu dalam proses web scraping digunakan modul beautifulsoup.

Pertama tama diimport dahulu modulnya, lalu string r diubah menjadi objek beautiful soup dan digunakan parser lxml

Jika di print soup nya, sekilas terlihat seperti string HTML, namun sebenarnya itu merupakan sebuah objek soup yang bisa ditelusri setiap elemennya!

In [ ]:
from bs4 import BeautifulSoup 

soup = BeautifulSoup(r,"lxml")
print(type(soup))
print(soup.prettify()[1:100])

### Dapatkan Semua Link

Untuk mendapatkan semua link kita dapat mencari semua tag html "a". Namun tentunya tidak semua hasilnya merupakan link artikel. perlu kita filter terlebih dahulu.

In [ ]:
links = soup.find_all('a')
#print(len(links))

#for link in links:
#    print('%s: %s'%(link.text.strip(),link["href"]))


Dapat dilihat link yang merujuk ke berita memiliki pola '/read/' dan benar-benar link bukan javascript maupun link '#'. Maka dari itu bisa kita filter, dan hasilnya bisa kita simpan di sebuah list

In [ ]:
berita_link = [link for link in links if '/read/' in str(link) and 'javascript:void(0)' not in str(link) and '#' not in str(link)]
#print(len(berita_link))
#for link in berita_link:
#    print('%s: %s'%(link.text.strip(),link["href"]))


### Dapatkan Isi Berita

Sebelum kita menelusuri satu per satu kita perlu tahu dahulu struktur dari sebuah halaman berita pada kompas.com untuk mendapatkan properties yang kita inginkan. 

Gunakan inspect element!

Misalkan disini ingin diambil judul berita, isi berita, beserta tanggal pembuatan berita. Setelah dianalisa, judul berada pada elemen div yang mempunyai class kcm-read-top, sedangkan isi_berita pada div yang mempunyai kelas kcm-read-text, dan tanggal div dengan class kcm-date. Untuk mendapatkan text atau inner html nya kita bisa memanfaatkan properti text.

Namun disini terdapat kendala karena tag tanggal tergabung dengan berbagai text lainnya jadi harus dilakukan pemrosesan terlebih dahulu. Cara yang akan digunakan disini sedikit berbahaya karena memanfaatkan separator yang bisa jadi tidak konsisten di semua berita, meskipun sampai saat ini belum ditemukan.

In [ ]:
html = urllib.urlopen("http://tekno.kompas.com/read/2016/04/26/19500067/6.Aplikasi.Gratis.Pengirit.Baterai.Android").read()
soup = BeautifulSoup(html, "lxml")

judul = soup.find("div","kcm-read-top").find("h2").text.strip()
isi_berita = soup.find("div","kcm-read-text").text.strip()
tanggal = soup.find("div","kcm-date").text

#print(tanggal.strip())
tanggal = tanggal.strip().split(',')[1].split('|')[0].strip()
#print(tanggal)

Sebagai tambahan, python juga dapat menyimpan sebuah gambar. Disini akan coba kita terapkan untuk menyimpan gambar dari berita. 

In [ ]:
gambar = soup.find("div","kcm-read-top").find("img")
#print(gambar['src'])

from PIL import Image
from StringIO import StringIO
import string

r = urllib.urlopen(gambar["src"]).read()
#print(r)

i = Image.open(StringIO(r))
exclude = set(string.punctuation)
judul = ''.join(ch for ch in judul if ch not in exclude)

nama_file = "images/" + judul + ".jpg"
i.save(nama_file,'JPEG')

### Gabungkan ke Dalam Fungsi

In [ ]:
def getBerita(link):
    html = urllib.urlopen(link).read()
    soup = BeautifulSoup(html, "lxml")

    judul = soup.find("div","kcm-read-top").find("h2").text.strip().encode("utf8")
    isi_berita = soup.find("div","kcm-read-text").text.strip().encode("utf8")
    tanggal = soup.find("div","kcm-date").text
    tanggal = tanggal.strip().split(',')[1].split('|')[0].strip()
    gambar = soup.find("div","kcm-read-top").find("img")
    
    r = urllib.urlopen(gambar["src"]).read()
    i = Image.open(StringIO(r))
    
    exclude = set(string.punctuation)
    judul = ''.join(ch for ch in judul if ch not in exclude)
    
    nama_file = "images/" + judul + ".jpg"
    i.save(nama_file,'JPEG')
    
    return [judul,isi_berita,tanggal]

### Dapatkan Semua Data dari Link Berita

In [ ]:
berita = []
for link in berita_link:
    print('%s: %s'%(link.text.strip(),link["href"]))
    berita.append(getBerita(link["href"]))

### Simpan ke dalam CSV

In [ ]:
import csv

with open('data_berita.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(berita)